In [1]:
DATA_PREPARATION_VOTE_METHOD = "max_vote_window" # "max_vote_window" or "sum_and_normalize". Decides how to aggregate the predictions of the overlapping windows
EXISTING_CHECKPOINT_KAGGLE_DATASET_ID = "hsm-models" # set to None if you want to train a new model on Kaggle. Else, set to the Kaggle dataset ID where the existing model checkpoints are stored
PRETRAINED_MODEL_NAME_OR_PATH = "inception_v3"

In [2]:
import os
import sys

from tqdm import tqdm
import torch

import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import DataLoader


if bool(os.environ.get("KAGGLE_URL_BASE", "")):
  import sys
  # running on kaggle
  sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
  # running locally
  sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

from src.utils.utils import get_raw_data_dir, get_processed_data_dir, get_submission_csv_path, get_models_save_path, set_seeds
from src.utils.constants import Constants

from src.datasets.multi_spectrogram import MultiSpectrogramDataset
from src.models.base_cnn import BaseCNN
from src.utils.eeg_spectrogram_creator import EEGSpectrogramGenerator

set_seeds(Constants.SEED)

2025-11-14 09:23:20,385 :: root :: INFO :: Initialising Utils
2025-11-14 09:23:22,224 :: root :: INFO :: Initialising Datasets
2025-11-14 09:23:23,341 :: root :: INFO :: Initialising Models


Skipping module tcn due to missing dependency: No module named 'pytorch_tcn'


/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened bec

In [3]:
DATA_PATH = get_raw_data_dir()
test_df = pd.read_csv(DATA_PATH / "test.csv")

In [4]:
test_df.head()

,spectrogram_id,eeg_id,patient_id
0,853520,3911565283,6885


In [5]:
class InfCFG:
    data_path = DATA_PATH
    model_name = PRETRAINED_MODEL_NAME_OR_PATH
    target_size = 6
    img_size = (128, 256)
    n_splits = 5
    batch_size = 64
    num_workers = 8
    model_dir = get_models_save_path(EXISTING_CHECKPOINT_KAGGLE_DATASET_ID) / "multi_spec_cnn" / PRETRAINED_MODEL_NAME_OR_PATH / DATA_PREPARATION_VOTE_METHOD
    test_eeg_spec_path = get_processed_data_dir() / "eeg_spectrograms" / "test" / "cwt"
    dropout_p = 0.1

InfCFG.model_paths = [os.path.join(InfCFG.model_dir, f'best_model_fold{i}.pth') for i in range(InfCFG.n_splits)]

In [6]:
InfCFG.test_eeg_spec_path.mkdir(parents=True, exist_ok=True)
print(f"Test EEG spectrogram path: {InfCFG.test_eeg_spec_path}")

existing_specs = len(list(InfCFG.test_eeg_spec_path.glob("*.npy")))

test_eeg_ids = test_df["eeg_id"].unique()

if existing_specs == len(test_eeg_ids):
  print("Test EEG spectrograms already exist. Skipping generation.")
else:
  spectrogram_creator = EEGSpectrogramGenerator(["cwt"])
  for eeg_id in tqdm(test_eeg_ids, desc="Generating EEG Spectrograms"):
      eeg_path = os.path.join(InfCFG.data_path, "test_eegs", f"{eeg_id}.parquet")
      eeg = pd.read_parquet(eeg_path)
      spectrograms = spectrogram_creator.generate(eeg)
      np.save(InfCFG.test_eeg_spec_path / f"{eeg_id}.npy", spectrograms['cwt'])

Test EEG spectrogram path: /home/david/git/aicomp/data/processed/eeg_spectrograms/test/cwt
Test EEG spectrograms already exist. Skipping generation.


In [7]:
def run_inference():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    test_df = pd.read_csv(os.path.join(InfCFG.data_path, 'test.csv'))
    test_dataset = MultiSpectrogramDataset(
        df=test_df, 
        targets=Constants.TARGETS, 
        data_path=InfCFG.data_path, 
        img_size=InfCFG.img_size, 
        eeg_spec_path=InfCFG.test_eeg_spec_path, 
        mode='test',
        apply_augmentations=False
    )
    test_loader = DataLoader(
        test_dataset, batch_size=InfCFG.batch_size, shuffle=False, num_workers=InfCFG.num_workers
    )

    all_fold_predictions = []

    for i, path in enumerate(InfCFG.model_paths):
        print(f"\n========== Inferencing with Fold {i} Model ==========")
        if not os.path.exists(path):
            print(f"Model file not found: {path}. Skipping this fold.")
            continue
            
        model = BaseCNN(InfCFG.model_name, pretrained=False, num_classes=InfCFG.target_size, dropout_p=InfCFG.dropout_p)
        model.load_state_dict(torch.load(path, map_location=device))
        model.to(device)
        model.eval()

        current_fold_preds = []
        with torch.no_grad():
            for images in tqdm(test_loader, desc=f"Predicting Fold {i}"):
                outputs = model(images.to(device))
                probs = F.softmax(outputs, dim=1).cpu().numpy()
                current_fold_preds.append(probs)
        
        all_fold_predictions.append(np.concatenate(current_fold_preds))

    if not all_fold_predictions:
        print("No models were found for inference. Aborting.")
        return

    avg_predictions = np.mean(all_fold_predictions, axis=0)
    submission = pd.DataFrame({"eeg_id": test_df["eeg_id"]})
    submission[Constants.TARGETS] = avg_predictions

    submission.to_csv(get_submission_csv_path(), index=False)

In [8]:
run_inference()

Using device: cuda

========== Inferencing with Fold 0 Model ==========


Predicting Fold 0: 100%|██████████| 1/1 [00:01<00:00,  1.38s/it]



========== Inferencing with Fold 1 Model ==========


Predicting Fold 1: 100%|██████████| 1/1 [00:00<00:00,  2.66it/s]



========== Inferencing with Fold 2 Model ==========


Predicting Fold 2: 100%|██████████| 1/1 [00:00<00:00,  2.93it/s]



========== Inferencing with Fold 3 Model ==========


Predicting Fold 3: 100%|██████████| 1/1 [00:00<00:00,  2.71it/s]



========== Inferencing with Fold 4 Model ==========


Predicting Fold 4: 100%|██████████| 1/1 [00:00<00:00,  2.87it/s]
